# Constructive heuristics comparison

The objective of the $\alpha$-neighbor $p$-center problem can be thought of as distributing the facilities among the clients to cover them efficiently, which is the actual goal of the $p$-center problem, so a constructive heuristic that uses its objective function will be tested and compared against a greedy heuristic that takes into account the objective function of this problem.

There will be used 20 random instances of size $n = 50$, $p = 5$ and other 20 of size $n = 400$, $p = 20$, and each one will be tested with both $\alpha = 2$ and $\alpha = 3$. The coordinates of the points are between 0 and 1000 for both planes.

In [2]:
from copy import deepcopy
from typing import List

from models.instance import Instance


def generate_instances(amount: int, n: int, p: int) -> List[Instance]:
    alpha2 = [
        Instance.random(n, p, 2, 1000, 1000)
        for _ in range(amount)
    ]
    alpha3 = deepcopy(alpha2)
    for i in alpha3:
        i.alpha = 3
    return alpha2 + alpha3

In [3]:
instances = generate_instances(20, 50, 5) + generate_instances(20, 400, 20)

We will use the following code to measure the time taken by the evaluations and the objective function results, formatted in a Pandas DataFrame.

In [4]:
import timeit

import pandas as pd

from heuristics.constructive import pdp_based, greedy
from utils import eval_obj_func

def measure(instance, heuristic):
    start = timeit.default_timer()
    solution = heuristic(instance)
    time = timeit.default_timer() - start
    of = eval_obj_func(instance, solution)
    return heuristic.__name__, solution, of, time

def get_dataframe(data):
    return pd.DataFrame({
        colname: [d[i] for d in data]
        for colname, i in zip(
            ('n', 'p', 'a', 'heuristic', 'solution', 'OF', 'seconds'),
            range(len(data[0])))
    })

## Comparing data

In [5]:
pdp_data = [(*i.get_parameters(), *measure(i, pdp_based))  for i in instances]

pdp_df = get_dataframe(pdp_data)
pdp_df

,n,p,a,heuristic,solution,OF,seconds
0,50,5,2,pdp_based,"{33, 5, 37, 40, 28}",628,0.000760
1,50,5,2,pdp_based,"{33, 2, 34, 13, 14}",607,0.000535
2,50,5,2,pdp_based,"{35, 5, 8, 9, 42}",794,0.000527
3,50,5,2,pdp_based,"{18, 19, 13, 45, 14}",611,0.000617
4,50,5,2,pdp_based,"{1, 7, 27, 29, 47}",643,0.000525
...,...,...,...,...,...,...,...
75,400,20,3,pdp_based,"{66, 68, 198, 74, 395, 397, 399, 16, 81, 209, ...",406,0.074006
76,400,20,3,pdp_based,"{261, 8, 201, 202, 13, 22, 281, 155, 222, 289,...",390,0.093924
77,400,20,3,pdp_based,"{1, 259, 325, 198, 134, 201, 204, 77, 78, 208,...",375,0.073400
78,400,20,3,pdp_based,"{391, 10, 13, 77, 271, 16, 17, 274, 144, 84, 2...",372,0.064396


Saving the evaluation to a CSV:

In [11]:
import os

OUT_FOLDER = 'nb_results\\constructive'
pdp_df.to_csv(os.path.join(OUT_FOLDER, 'pdp_df.csv'))

Calculating statistics of the results:

In [12]:
stats = {
    'pdp': {
        f'n{n}': {
            f'a{alpha}': pdp_df[
                (pdp_df['n'] == n) &
                (pdp_df['a'] == alpha)
                ][['OF', 'seconds']].describe()
            for alpha in (2, 3)
        }
        for n in (50, 400)
    }
}

In [16]:
stats['pdp']['n50']['a2'], stats['pdp']['n50']['a3']

(               OF    seconds
 count   20.000000  20.000000
 mean   700.400000   0.000871
 std     88.983381   0.000701
 min    565.000000   0.000502
 25%    627.750000   0.000527
 50%    678.500000   0.000591
 75%    765.500000   0.000941
 max    881.000000   0.003575,
                OF    seconds
 count   20.000000  20.000000
 mean   843.050000   0.000859
 std     71.258961   0.000333
 min    738.000000   0.000509
 25%    780.750000   0.000562
 50%    838.000000   0.000886
 75%    906.750000   0.001055
 max    958.000000   0.001752)